## Connect to an ArcGIS Online Organization (as administrator)

In [1]:
from arcgis.gis import GIS
import getpass

password = getpass.getpass("Enter password: ")
g = GIS('https://www.arcgis.com', 'johnyHack', password)
print("Connected to: {}\nConnected as: {}".format(g.properties.urlKey + "." + g.properties.customBaseUrl,
                                                 g.users.me.username))

Enter password: ········
Connected to: esrihax.maps.arcgis.com
Connected as: johnyHack


## Create a list of Feature Layer items

In [2]:
fl_srch= g.content.search(query="* AND NOT owner:esri*", item_type="Feature Layer", max_items=200)

In [3]:
print("This ArcGIS Online Organization contains {} feature layers.".format(len(fl_srch)))

This ArcGIS Online Organization contains 176 feature layers.


### Alternatives to list the Feature Layer `itemId` values

#### Print out a list of `itemId` values for those Layers that are hosted.

In [4]:
import requests

#for i in range (len(fl_srch)):
for i in range(0, 15):
    if not fl_srch[i].layers is None:
        for lyrs in fl_srch[i].layers:
            try:
                if requests.get(lyrs.url).status_code == 200:
                    if not 'arcgis.com' in lyrs.url:
                        print("{:<15}{}".format("\tReferenced:", lyrs.properties.name))
                    else:
                        print("{:<65}{}".format(lyrs.properties.name, lyrs.properties.serviceItemId))
                else:
                    print("{:<2}{}\n{:<5}{}".format("", "Cannot reach service: ", "", lyrs.url))
            except OSError as oe:
                print("{:<2}{:<22}{}".format("", "Connection Error", lyrs.url))       

LA_COUNTY_CITYBOUNDARIES                                         52652df1c26245dc96987963c62032d4
_HRR_Blank                                                       33d5ce25164941099ece79832e16b3d4
Pronto_Bike_Share_Stations2                                      3c692805d6b347ca8b8b11382da2b3d3
HotSpots_Teen_Mot_1                                              6f93c95f46134dec9ed72b7ea101773a
BufferedFeatures                                                 1a81776d28774c53affb2560d38a7219
Pasadena_Council_Districts                                       e19c954a83d54759853cb6a5831d6e0f
	Referenced:   Zoning
  Cannot reach service: 
     https://gisrevprxy.seattle.gov/ArcGIS/rest/services/SDOT_EXT/DSG_DATASHARING/MapServer/66
SFO Spaces                                                       80828bbbe89a4079b6780e218336118b
PasadenaUSD                                                      df4a8d03fb3c408884820df8571a44a6
Pasadena_Citizen_Service_Requests                                d3d059978

#### Use `urllib.parse` to parse the feature layer item url into components and retrieve the hosted  `itemId` values.

In [5]:
from urllib.parse import urlparse

for fl_item in fl_srch[:20]:
    purl = urlparse(fl_item.url)
    if 'arcgis.com' in purl.netloc:
        if fl_item.layers is None:
            print("{:<65}{}".format(fl_item.title, fl_item.id))
        else:
            for flyr in fl_item.layers:
                print("{:<65}{}\n{:<2}{}".format(flyr.properties['name'][:64], 
                                                 flyr.properties['serviceItemId'], 
                                                 "", 
                                                 flyr.url))

LA_COUNTY_CITYBOUNDARIES                                         52652df1c26245dc96987963c62032d4
  http://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/LA_COUNTY_CITYBOUNDARIES/FeatureServer/0
_HRR_Blank                                                       33d5ce25164941099ece79832e16b3d4
  https://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/Hospital_Referral_Regions/FeatureServer/0
Pronto_Bike_Share_Stations2                                      3c692805d6b347ca8b8b11382da2b3d3
  http://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/Pronto_Bike_Share_Stations/FeatureServer/0
HotSpots_Teen_Mot_1                                              6f93c95f46134dec9ed72b7ea101773a
  http://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/Hot Spots Teen_Mot_1/FeatureServer/0
BufferedFeatures                                                 1a81776d28774c53affb2560d38a7219
  http://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/Buffer

## Create a list of Web Map items and use it for various searches.

In [7]:
wm_items = g.content.search("*", item_type="Web Map", max_items=200)

In [23]:
print("This ArcGIS Online Organization contains {} web maps.".format(len(wm_items)))

This ArcGIS Online Organization contains 159 web maps.


### Loop through all the web maps and print the layer `title`  abd `url` for each layer:

#### Method 1:

In [ ]:
from arcgis.mapping import WebMap

for wm in wm_items[:30]:
    wmo = WebMap(wm)
    print("{}\n{:-<75}".format(wm.title, ""))
    for lyr in wmo.layers:
        print("\t{}\n\t{}".format(lyr.title, lyr.url))

### Loop through all the web maps and search for a specific `itemId`  

In [9]:
from arcgis.mapping import WebMap

for wm in wm_items:
    pwm = WebMap(wm)
    for lyr in pwm.layers:
        if hasattr(lyr, 'itemId'):
            if lyr.itemId == '80828bbbe89a4079b6780e218336118b':
                print("{} is in:\n{:2}{} Web Map".format(lyr.title,"", wm.title))

Spaces_T2 - SFO Spaces is in:
  Wearable World Invent Web Map


### Obtain a web map `Item` from the list of web map items, and instantiate a `WebMap` object from the item. Then loop through the web map `layers` to print out the name, id, and layer url values.

In [10]:
wbm = wm_items[88]
wbm2 = WebMap(wbm)
print("Web Map: {}\n{:<2}{:<40}\t{:<40}\n{:-<40}\t{:-<40}".format(wbm.title, "", "Layer","Layer Id", "", ""))
for lyr in wbm2.layers:
    if hasattr(lyr, 'itemId'):
        if hasattr(lyr, 'url'):
            print("{:<40}\t{}\n{:<2}{}".format(lyr.title[0:38], lyr.itemId, "", lyr.url))
        else:
            print("{:<40}\t{}".format(lyr.title[0:38], lyr.itemId))
    elif hasattr(lyr, 'id'):
        if hasattr(lyr, 'url'):
            print("{:<40}\t{}\n{:<2}{}".format(lyr.title[0:38], lyr.id, "", lyr.url))
        else:
            print("{:<40}\t{}".format(lyr.title[:38], lyr.id))
    else:
        print("Layer has no itemId or id.")

Web Map: Snow / Ice Problems
  Layer                                   	Layer Id                                
----------------------------------------	----------------------------------------
Snow / Ice Complaint                    	458ee20af23d4a72b1e91045c08fcacb
  https://services1.arcgis.com/rxMuAi9NyHDn2zKX/arcgis/rest/services/SnowIceProblemReports/FeatureServer/0


## Search for specific Layers in Web Maps

In [11]:
subfls = g.content.search(query='Toronto *', item_type='Feature Layer', max_items=200)
subfls

[<Item title:"Toronto" type:Feature Layer Collection owner:johnyHack>]

In [12]:
for flc in subfls:
    if flc.layers:
        for flyr in flc.layers:
            print(flyr.properties.name)

Awesome Spots
Streets to Checkout
Fun Neighborhood


In [13]:
new_wm = wm_items[10:20]
new_wm

[<Item title:"LA Bus BDS Aggregation" type:Web Map owner:mpilouk_ze>,
 <Item title:"Flight-Enter-Exit-DC" type:Web Map owner:mpHackathon>,
 <Item title:"CalTrans Freeway Traffic Sensors" type:Web Map owner:mpHackathon>,
 <Item title:"Toronto Stuff-Copy" type:Web Map owner:johnyHack>,
 <Item title:"Esri Indoor Tracks - S4-2" type:Web Map owner:gtieman_esrihax>,
 <Item title:"Trying_Collector_in_Zurich" type:Web Map owner:johnyHack>,
 <Item title:"Illinois Licensed Swimming Facilities" type:Web Map owner:johnyHack>,
 <Item title:"Alameda Networkfleet-GeoEvent Demo-4" type:Web Map owner:alamedaEsri>,
 <Item title:"Nursing Homes CCA SumSkill" type:Web Map owner:johnyHack>,
 <Item title:"Workforce Starter_workers" type:Web Map owner:johnyHack>]

In [14]:
for nwm in new_wm:
    try:
        wmurls = [lyr.url for lyr in WebMap(nwm).layers]
    except AttributeError as ae:
        pass
    for flitem in subfls:
        if flitem.layers:
            for flyr in flitem.layers:
                try:
                    if flyr.url in wmurls:
                        print("{} is in {} Web Map.".format(flyr.properties.name, nwm.title))
                except AttributeError as ae:
                    pass
        else:
            pass

Awesome Spots is in Toronto Stuff-Copy Web Map.
Streets to Checkout is in Toronto Stuff-Copy Web Map.
Fun Neighborhood is in Toronto Stuff-Copy Web Map.


## Method prior to the Web Map object

#### Use the `content` property (the `ContentManager`) on the gis to search for all organization Web Maps 

In [ ]:
wmaps = g.content.search("*", item_type="Web Map", max_items = 200, outside_org=False)

#### Loop through all the web maps, using the `get_data()` function to parse the returned json for the `operationalLayers` in each web map. Then loop through the layers in each web map and return each layer's `title` and `itemId` value:

In [ ]:
for wbmap in wmaps:
    print("{}".format(wbmap.title))
    print("{:*<90}".format(""))
    raw_data = wbmap.get_data()
    lyrs = raw_data['operationalLayers']
    for lyr in lyrs:
        try:
            if 'itemId' in lyr.keys():
                print("\t{:<50}{}".format(lyr['title'][:45], lyr['itemId']))
            else:
                print("\t{:<50}{}".format(lyr['title'][:45], lyr['id']))
        except:
            pass
    print("\n")

#### Retrieve a layer `itemId` and search the org for which web maps contain that layer.

In [ ]:
id_of_interest = '20a29db43f11408db2e6088b30740363'

In [ ]:
lyr_itm = g.content.get(id_of_interest)
lyr = lyr_itm.layers[0]
print("{} is in the following Web Maps:".format(lyr.properties.name))
print("{:-<50}".format(""))
lyr = None
for wm in wmaps:
    raw_data = wm.get_data()
    lyrs = raw_data['operationalLayers']
    for lyr in lyrs:
        if 'itemId' in lyr.keys() and lyr['itemId'] == id_of_interest:
            print("\t" + wm.title)